In [23]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
import xgboost as xgb
import numpy as np

In [3]:
import os
path = os.path.dirname(xgb.__file__)
print(path)

/home/agavrilenko/anaconda3/lib/python3.6/site-packages/xgboost


In [4]:
#help(xgboost.training.train)
help(xgb.train)

Help on function train in module xgboost.training:

train(params, dtrain, num_boost_round=10, evals=(), obj=None, feval=None, maximize=False, early_stopping_rounds=None, evals_result=None, verbose_eval=True, xgb_model=None, callbacks=None, learning_rates=None)
    Train a booster with given parameters.
    
    Parameters
    ----------
    params : dict
        Booster params.
    dtrain : DMatrix
        Data to be trained.
    num_boost_round: int
        Number of boosting iterations.
    evals: list of pairs (DMatrix, string)
        List of items to be evaluated during training, this allows user to watch
        performance on the validation set.
    obj : function
        Customized objective function.
    feval : function
        Customized evaluation function.
    maximize : bool
        Whether to maximize feval.
    early_stopping_rounds: int
        Activates early stopping. Validation error needs to decrease at least
        every **early_stopping_rounds** round(s) to cont

In [14]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xgb

In [15]:
# load data
import pandas as pd
df_usa = pd.read_csv('data/houses/housesalesprediction/kc_house_data.csv', delimiter=",")

dataset.head()


df_usa.drop(['id', 'date'], axis=1, inplace=True)
X = df_usa.drop("price",axis=1).values
y = df_usa["price"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

# split data into X and y
# X = dataset[:,0:8]
# Y = dataset[:,8]
# # split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=6)

In [75]:
dtrain = xgb.DMatrix(data = X_train, label = y_train, silent= True)

deval = xgb.DMatrix(data = X_test, label = y_test, silent= True)
# evals_list = []
# evals_list.append((deval, "eval matrix"))
num_epochs = 600
params = {}
params['num_boost_round'] = num_epochs
params['early_stopping_rounds'] = 1000
#params['verbose_eval'] = 1
params['eval_metric'] = 'rmse'
params['evals_result'] = {}
params['max_depth'] = 3




In [91]:
learning_rates = []
tresholds = []
learning_rates.append(0.1)
counter = 0
best = []
progress = dict()

def reduceLRonPlateau(i,n):

    factor = 0.2
    min_lr = 1e-30
    min_delta = 1e-10
    patience = 0
    verbose = 0
    cooldown = 3
    cooldown_counter = 0  # Cooldown counter.
    wait = 0
    #best = 0
    mode = 'min'  
    monitor_op = None
    old_lr = learning_rates[0]
    new_lr = learning_rates[0] 
    
    def in_cooldown():
            return cooldown_counter > 0
    
    def _reset(mode):
        """Resets wait counter and cooldown counter."""
#         if mode not in ['auto', 'min', 'max']:
#             warnings.warn('Learning Rate Plateau Reducing mode %s is unknown, '
#                           'fallback to auto mode.' % (mode),
#                           RuntimeWarning)
#             mode = 'auto'
#         if (mode == 'min' or
#            (mode == 'auto' and 'acc' not in monitor)):
#             monitor_op = lambda a, b: np.less(a, b - min_delta)
#             best = np.Inf
#         else:
#             monitor_op = lambda a, b: np.greater(a, b + min_delta)
#             best = -np.Inf
            
        cooldown_counter = 0
        wait = 0
        return wait, cooldown_counter
    
    def checker(wait,best, new_lr, learning_rates):
        if (i == 0):
            wait, counter = _reset(mode)
            print("initialization...")
            print("lr: ", learning_rates[0])
            best.append(100000000)
            
        if (i>0):
            old_lr = learning_rates[i-1]
            #new_lr = learning_rates[i-1]
            current = progress['eval']['rmse'][i-1]
            if in_cooldown():
                cooldown_counter -= 1
                wait = 0
              
            print("iter: ", i, " Current_loss: ", current, "best: ", best[i-1])
            if np.less(current, best[i-1] - min_delta):
                best.append(current)
                wait = 0
             
            elif not in_cooldown():
                #rint("not in_cooldown")
                best.append(best[-1])
                wait += 1
                if wait >= patience:
                    
                    if old_lr > min_lr:
                        new_lr = old_lr * factor
                        new_lr = max(new_lr, min_lr)
                        print("iter: ", i, "reducing lr -- ", " old_lr: ", old_lr, " new_lr: ", new_lr)
                        
#                         if self.verbose > 0:
#                             print('\nEpoch %05d: ReduceLROnPlateau reducing '
#                                   'learning rate to %s.' % (epoch + 1, new_lr))
                        cooldown_counter = cooldown
                        wait = 0
        return new_lr


       # print("step ",progress['train']['logloss'][i-1], "i = ", i)

            
    
    new_lr = checker(wait,best,new_lr,learning_rates)
    learning_rates.append(new_lr)
        
    return new_lr

In [92]:

def run_gbm(dtrain, dvalid, param, evals_result, learning_rate):
    # check training arguments in param
    n_round = param.get('num_boost_round', 100)
    early_stop = param.get('early_stopping_rounds', 0)
    verbose_eval = param.get('verbose_eval', 50)
    # specify validations set to watch performance
    watchlist = [(dtrain,'train') ,(deval,'eval')]
    #callbacks_list = [learning_rates]

    bst = xgb.train(params=param,
            dtrain=dtrain,
            num_boost_round=n_round,
            evals=watchlist,
            early_stopping_rounds=early_stop,
            verbose_eval = verbose_eval,
            evals_result = evals_result, 
            callbacks = [xgb.callback.reset_learning_rate(reduceLRonPlateau)])
                   
    return bst 

In [ ]:
model1 = run_gbm(dtrain, deval, params,progress, learning_rate = 0.1)

initialization...
lr:  0.1
[0]	train-rmse:597658	eval-rmse:590266
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 1000 rounds.
iter:  1  Current_loss:  590265.5 best:  100000000
iter:  2  Current_loss:  539130.9375 best:  590265.5
iter:  3  Current_loss:  493740.15625 best:  539130.9375
iter:  4  Current_loss:  453324.34375 best:  493740.15625
iter:  5  Current_loss:  417079.59375 best:  453324.34375
iter:  6  Current_loss:  385368.84375 best:  417079.59375
iter:  7  Current_loss:  357464.09375 best:  385368.84375
iter:  8  Current_loss:  332269.125 best:  357464.09375
iter:  9  Current_loss:  310484.65625 best:  332269.125
iter:  10  Current_loss:  290920.0 best:  310484.65625
iter:  11  Current_loss:  273306.21875 best:  290920.0
iter:  12  Current_loss:  258614.453125 best:  273306.21875
iter:  13  Current_loss:  245829.46875 best:  258614.453125
iter:  14  Current_loss:  233888.078125 best:  245829.

iter:  134  Current_loss:  121107.046875 best:  121259.726562
iter:  135  Current_loss:  121011.234375 best:  121107.046875
iter:  136  Current_loss:  121005.46875 best:  121011.234375
iter:  137  Current_loss:  120957.53125 best:  121005.46875
iter:  138  Current_loss:  120903.921875 best:  120957.53125
iter:  139  Current_loss:  120814.71875 best:  120903.921875
iter:  140  Current_loss:  120776.023438 best:  120814.71875
iter:  141  Current_loss:  120747.125 best:  120776.023438
iter:  142  Current_loss:  120673.234375 best:  120747.125
iter:  143  Current_loss:  120617.757812 best:  120673.234375
iter:  144  Current_loss:  120559.195312 best:  120617.757812
iter:  145  Current_loss:  120514.46875 best:  120559.195312
iter:  146  Current_loss:  120392.351562 best:  120514.46875
iter:  147  Current_loss:  120090.507812 best:  120392.351562
iter:  148  Current_loss:  120051.8125 best:  120090.507812
iter:  149  Current_loss:  120039.710938 best:  120051.8125
iter:  150  Current_loss: 

iter:  205  Current_loss:  119445.265625 best:  119412.53125
iter:  205 reducing lr --   old_lr:  1.0485760000000013e-15  new_lr:  2.0971520000000026e-16
iter:  206  Current_loss:  119445.265625 best:  119412.53125
iter:  206 reducing lr --   old_lr:  2.0971520000000026e-16  new_lr:  4.1943040000000054e-17
iter:  207  Current_loss:  119445.265625 best:  119412.53125
iter:  207 reducing lr --   old_lr:  2.0971520000000026e-16  new_lr:  4.1943040000000054e-17
iter:  208  Current_loss:  119445.265625 best:  119412.53125
iter:  208 reducing lr --   old_lr:  4.1943040000000054e-17  new_lr:  8.388608000000012e-18
iter:  209  Current_loss:  119445.265625 best:  119412.53125
iter:  209 reducing lr --   old_lr:  4.1943040000000054e-17  new_lr:  8.388608000000012e-18
iter:  210  Current_loss:  119445.265625 best:  119412.53125
iter:  210 reducing lr --   old_lr:  8.388608000000012e-18  new_lr:  1.6777216000000024e-18
iter:  211  Current_loss:  119445.265625 best:  119412.53125
iter:  211 reducin

iter:  263  Current_loss:  119210.914062 best:  119169.015625
iter:  263 reducing lr --   old_lr:  0.00016000000000000007  new_lr:  3.200000000000001e-05
iter:  264  Current_loss:  119210.859375 best:  119169.015625
iter:  264 reducing lr --   old_lr:  3.200000000000001e-05  new_lr:  6.400000000000003e-06
iter:  265  Current_loss:  119210.851562 best:  119169.015625
iter:  265 reducing lr --   old_lr:  3.200000000000001e-05  new_lr:  6.400000000000003e-06
iter:  266  Current_loss:  119210.859375 best:  119169.015625
iter:  266 reducing lr --   old_lr:  6.400000000000003e-06  new_lr:  1.2800000000000007e-06
iter:  267  Current_loss:  119210.859375 best:  119169.015625
iter:  267 reducing lr --   old_lr:  6.400000000000003e-06  new_lr:  1.2800000000000007e-06
iter:  268  Current_loss:  119210.859375 best:  119169.015625
iter:  268 reducing lr --   old_lr:  1.2800000000000007e-06  new_lr:  2.560000000000001e-07
iter:  269  Current_loss:  119210.859375 best:  119169.015625
iter:  269 reduc

iter:  316  Current_loss:  119210.859375 best:  119169.015625
iter:  316 reducing lr --   old_lr:  2.1474836480000036e-23  new_lr:  4.2949672960000076e-24


In [ ]:
# fit model no training data
model = XGBRegressor(n_estimators=num_epochs, learning_rate=0.1)
eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(X_train, y_train, eval_metric=["error", "rmse"], eval_set=eval_set, verbose=False)

In [ ]:
# retrieve performance metrics
results = model.evals_result()
#results1 = model1.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(25, epochs)

# plot log loss
fig, ax = pyplot.subplots(figsize=(10, 10))
ax.plot(x_axis, results['validation_0']['rmse'][25::], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'][25::], label='Eval')
ax.plot(x_axis, progress['train']['rmse'][25::], label = 'customTrain')
ax.plot(x_axis, progress['eval']['rmse'][25::], label = 'customEval')
ax.legend()
pyplot.ylabel('RMSE')
pyplot.title('XGBoost RMSE')

pyplot.show()
print("min customEval:", min(progress['eval']['rmse']))
print("min Eval:", min(results['validation_1']['rmse']))

In [ ]:
# plot classification error
# fig, ax = pyplot.subplots()
# ax.plot(x_axis, results['validation_0']['error'][25::], label='Train')
# ax.plot(x_axis, results['validation_1']['error'][25::], label='Test')
# ax.legend()
# pyplot.ylabel('Classification Error')
# pyplot.title('XGBoost Classification Error')
# pyplot.show()